In [ ]:
import babypandas as bpd
import numpy as np

# Lecture 6 –  More Querying and GroupBy
## DSC 10, Fall 2021

### Announcements

- Homework 2 is due on **Saturday 10/9 at 11:59pm**.
- Lab 3 is due on **Thursday 10/14 at 11:59pm**.
- If you've been attending the 3-4PM discussion section, consider attending the 4-5PM one instead – it's far emptier, meaning you'll get more support!

### Agenda

- Writing queries with multiple conditions.
- GroupBy.
- Challenge problems.

**Resources**: 
- [Resources tab of the course website](https://dsc10.com/resources/)
- [DSC 10 Reference Sheet](https://drive.google.com/file/d/1mQApk9Ovdi-QVqMgnNcq5dZcWucUKoG-/view)
- [BabyPandas Documentation](https://babypandas.readthedocs.io/en/latest/index.html)

## Queries with multiple conditions

### Recap: queries

- A "query" is code that extracts rows from a table for which certain condition(s) are true.
- Below we load in our NBA salaries 🏀 dataset from last time.

In [ ]:
salaries = bpd.read_csv('data/nba_salaries.csv').set_index('PLAYER')
salaries

### Question: Who was the highest paid center (C)?

In [ ]:
salaries.get('POSITION') == 'C'

In [ ]:
centers_only = salaries[salaries.get('POSITION') == 'C']
centers_only

In [ ]:
centers_only.sort_values(by='2015_SALARY', ascending=False).index[0]

### Boolean indexing

To select only some rows of `salaries`:

1. Make a list/array/Series of `True`s (keep) and `Falses` (toss).
2. Then pass it into `salaries[]`.

Rather than making the list by hand, we usually generate it by making a comparison.

### Elementwise comparisons

There are several types of comparisons we can make.

|symbol|meaning|
|--------|--------|
|`==` |equal to |
|`!=` |not equal to |
|`<`|less than|
|`<=`|less than or equal to|
|`>`|greater than|
|`>=`|greater than or equal to|

In [ ]:
# Find all players whose salary is less than or equal to 10 million
salaries[salaries.get('2015_SALARY') <= 10]

### Multiple conditions

- To do a query with multiple conditions, use `&` for "and" and `|` for "or".
- **Must use parentheses around each query.**
- Note: Don't use the Python keywords `and` and `or` here! They do not behave as you'd want.
    - Read the textbook for an explanation.

In [ ]:
# Find all the players who are guards (shooting guards or point guards)
salaries[(salaries.get('POSITION') == "PG") | (salaries.get('POSITION') == "SG")]

### The `&` and `|` operators work element-wise

In [ ]:
(salaries.get('POSITION') == "PG")

In [ ]:
(salaries.get('POSITION') == "SG")

In [ ]:
(salaries.get('POSITION') == "PG") | (salaries.get('POSITION') == "SG")

### More examples

In [ ]:
# Find all players on the Sacramento Kings who have a salary of over 8 million
salaries[(salaries.get('TEAM') == 'Sacramento Kings') & (salaries.get('2015_SALARY') > 8)]

### Discussion Question

Which of these queries evaluates to the number of centers **not** named Dwight Howard?

A. `salaries[(salaries.get('POSITION') == 'C') & (salaries.get('PLAYER') != 'Dwight Howard')].shape[0]`

B. `salaries[(salaries.get('POSITION') == 'C') and (salaries.index != 'Dwight Howard')].shape[0]`

C. `salaries[(salaries.get('POSITION') == 'C') & (salaries.index != 'Dwight Howard')].shape[0]`

D. `salaries[(salaries.get('POSITION') == 'C') & (salaries.index != 'Dwight Howard')].shape[1]`

### To answer, go to **[menti.com](https://menti.com)** and enter the code **1217 9449**.

## GroupBy: Split, Aggregate, Combine

### Motivating Question: Which team had the highest payroll?

- We can find the payroll for any one team.
    - For the Warriors, `salaries[salaries.get('TEAM') == 'Golden State Warriors'].get('2015_SALARY').sum()`.
- But how can we find the payroll for **all** teams at once?

In [ ]:
salaries

In [ ]:
salaries[salaries.get('TEAM') == 'Golden State Warriors'].get('2015_SALARY').sum()

In [ ]:
salaries[salaries.get('TEAM') == 'Los Angeles Lakers'].get('2015_SALARY').sum()

In [ ]:
salaries[salaries.get('TEAM') == 'Brooklyn Nets'].get('2015_SALARY').sum()

It seems like there has to be a better way. And there is!

### `.groupby`

Observe what happens when we use the `.groupby` method on `salaries`:

In [ ]:
salaries.groupby('TEAM').sum()

### An illustrative example

Consider the DataFrame `pets` containing pet species, colors, and weights.

| | **Species** | **Color** | **Weight** |
| --- | --- | --- | --- |
| **0** | dog | black | 40 |
| **1** | cat | golden | 15 |
| **2** | cat | black | 20 |
| **3** | dog | white | 80 |
| **4** | dog | golden | 25 |
| **5** | hamster | golden | 1 |

When we run `pets.groupby('Species').mean()`, `babypandas` internally creates three mini-DataFrames, one for each unique value of `'Species'`.

| | **Species** | **Color** | **Weight** |
| --- | --- | --- | --- |
| **0** | dog | black | 40 |
| **3** | dog | white | 80 |
| **4** | dog | golden | 25 |

---

| | **Species** | **Color** | **Weight** |
| --- | --- | --- | --- |
| **1** | cat | golden | 15 |
| **2** | cat | black | 20 |

---

| | **Species** | **Color** | **Weight** |
| --- | --- | --- | --- |
| **5** | hamster | golden | 1 |

It then takes the `mean` of every column (other than `Species` itself) that it can:

| | **Species** | **Color** | **Weight** |
| --- | --- | --- | --- |
| **0** | dog | black | 40 |
| **3** | dog | white | 80 |
| **4** | dog | golden | 25 |

<center>Mean Weight: (40 + 80 + 25) / 3 = 48.333</center>

---

| | **Species** | **Color** | **Weight** |
| --- | --- | --- | --- |
| **1** | cat | golden | 15 |
| **2** | cat | black | 20 |

<center>Mean Weight: (15 + 20) / 2 = 17.5</center>

---

| | **Species** | **Color** | **Weight** |
| --- | --- | --- | --- |
| **5** | hamster | golden | 1 |

<center>Mean Weight: (1) / 1 = 1</center>

Finally, it combines these means back into a single table, whose labels are equal to the unique values from the `Species` column, listed in alphabetical order.

| **Species** | **Weight** <br><br><br><br> |
| --- | --- |
| **cat** | 17.5 |
| **dog** | 48.33 |
| **hamster** | 1 |

These steps are: Split, Aggregate, Combine.

### Let's try it out!

In [ ]:
pets = bpd.DataFrame().assign(
    Species=['dog', 'cat', 'cat', 'dog', 'dog', 'hamster'],
    Color=['black', 'golden', 'black', 'white', 'golden', 'golden'],
    Weight=[40, 15, 20, 80, 25, 1]
)

pets

In [ ]:
pets.groupby('Species').mean()

### Back to the NBA...

In [ ]:
salaries

In [ ]:
salaries.groupby('TEAM').sum()

### `.groupby` in general

1. Use `.groupby(column_name)` to gather rows which have the same value in the specified column (`column_name`).
    - In the previous example, we grouped by `TEAM`.
    - In the resulting DataFrame, there will be one row for every unique value of `TEAM`.
2. Apply an **aggregation** function **within** each group.
    - In the previous example, we applied the `sum` method to every `TEAM`.
    - The aggregation is applied individually to each column.

Some aggregation functions: `.count()`, `.sum()`, `.mean()`, `.median()`, `.max()`, `.min()`.

### Observation #1

- The index has changed to team names.
- In general, the new row labels are the *group labels* (the thing that is used to determine if two rows were in same group).

In [ ]:
salaries

In [ ]:
salaries.groupby('TEAM').sum()

### Observation #2

- The `POSITION` and `TEAM` columns have disappeared. Why?

In [ ]:
salaries.groupby('TEAM').sum()

### Disappearing columns

- The aggregation function is applied to each column.
- If it doesn't make sense to apply it to a particular column, that column will disappear.
- Example: We can't sum strings, like in the `'POSITION'` column.
- Surprising example: *can* `max()` strings. How?

In [ ]:
salaries.groupby('TEAM').max() # Can you guess how the max position is determined?

### Other aggregation functions

- Aggregation functions work on arrays/lists/Series and return a single number.
- The most common ones are `.count()`, `.sum()`, `.mean()`, `.median()`, `.max()`, and `.min()`.


In [ ]:
salaries.groupby('TEAM').count()

In [ ]:
salaries.groupby('TEAM').mean()

In [ ]:
salaries.groupby('TEAM').max()

### Question: Which team had the highest payroll?

Strategy:
1. Group by `TEAM`.
2. Within each group, compute the total salary.
3. Sort by salary in descending order.
4. Take the name of the first team.

### Question: How many players play each position?

### Aside: Better column names?

To rename a column:

1. Add a new column with `.assign` containing the same values as the old column(s).
2. Drop the old column(s) with `.drop(columns=list_of_column_labels)`.

## Example: IMDB Dataset

In [ ]:
imdb = bpd.read_csv('data/imdb.csv').set_index('Title')
imdb

### Question: How many movies appear from each decade?

In [ ]:
imdb.groupby('Decade').count()

In [ ]:
imdb.groupby('Decade').count().plot(y='Year');

### Question: What was the highest rated movie of the 1990's?

Let's try and do this two different ways.

#### Without grouping

In [ ]:
imdb[imdb.get('Decade') == 1990].sort_values('Rating', ascending=False).index[0]

#### With grouping

In [ ]:
imdb.reset_index().groupby('Decade').max()

- It turns out that this method **does not** yield the correct answer. 
- When we use an aggregation function (e.g. `.max`), aggregation is done to each column individually. 
- While it's true that the highest rated movie from the 1990s has a rating of 9.2, that movie is **not** Unforgiven – instead, Unforgiven is the movie that's the latest in the alphabet among all movies from the 1990s.
- Grouping is not helpful here.

## Combining queries and groupby

### In other words, more challenging problems!

We'll try to cover these problems in class, but if we don't get to them all, we'll post their solutions [in this post on Campuswire](https://campuswire.com/c/G9636FFCF/feed/70) a few days after lecture.

### Question: How many years have more than 3 movies rated above 8.5?

In [ ]:
...

#### Aside: Using `.sum()` on a boolean array

In [ ]:
...

### Question: Out of the years with more than 3 movies, which had the highest average rating?

In [ ]:
...

### Question: Which year had the longest movie titles, on average?

**Hint:** Use `.str.len()` on the column or index that contains the names of the movies.

In [ ]:
...

### Question: What is the average rating of movies from years that had at least 3 movies in the list?

In [ ]:
...

## Summary

### Summary

- We can write queries that involve multiple conditions, as long as we:
    - Put parentheses around both conditions.
    - Separate conditions using `&` if you require both to be true, or `|` if you require at least one to be true.
- The method call `df.groupby(column_name).name_of_function()` **aggregates** all rows with the same value for `column_name` into a single row in the resulting table, according to the aggregation function `name_of_function`.
    - Aggregation functions we've seen include `.count()`, `.sum()`, `.mean()`, `.median()`, `.max()`, and `.min()`.
- **Next time:** A picture is worth a 1000 words – it's time to visualize!